# Welcome to the Kitchen Aide Agent 🐝


🎯 Scenario: You are planning for your next meal and for grocery shooping. Startin with an inventory of the foods available in your pantry and refridgerator and your favorite recipes, you may make several different queries of this agent to find:
- Which of your peronsal recipes you have the ingerdients to prepare
- If you do not have sufficient ingredients to make a personal recipe, what other recipes could be made with the on hand ingredients
- What ingredients you will need ot add to your shopping list to make a particular recipe

## 🔧 Setup
First, let's install the BeeAI Framework and set up our environment.

- setting up the observability so we can capture and log the actions our agent takes
- getting the "internal documents"

In [ ]:
%pip install -Uqq arize-phoenix s3fs unstructured "requests==2.32.4"\
 "opentelemetry-api==1.37.0" "opentelemetry-sdk==1.37.0" \
 "openinference-instrumentation-beeai==0.1.13" \
 "beeai-framework[duckduckgo,rag]" "fsspec==2025.3.0" jedi

# The following wraps Notebook output
from IPython.display import HTML, display
def set_css(*_, **__):
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))
get_ipython().events.register("pre_run_cell", set_css)

Now let's import the necessary modules:


In [ ]:
import os
import asyncio
import time
import phoenix as px
import ipywidgets
from typing import Any, Optional
from datetime import date
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.types import RequirementAgentOutput
from beeai_framework.agents.requirement.requirements import Requirement, Rule
from beeai_framework.agents.requirement.requirements.conditional import ConditionalRequirement
from beeai_framework.backend import ChatModel, ChatModelParameters
from beeai_framework.backend.document_loader import DocumentLoader
from beeai_framework.backend.embedding import EmbeddingModel
from beeai_framework.backend.text_splitter import TextSplitter
from beeai_framework.backend.vector_store import VectorStore
from beeai_framework.context import RunContext
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.middleware.trajectory import GlobalTrajectoryMiddleware
from beeai_framework.tools import Tool, ToolRunOptions, tool, StringToolOutput
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool
from beeai_framework.tools.search.retrieval import VectorStoreSearchTool
from beeai_framework.tools.think import ThinkTool
from beeai_framework.tools.weather import OpenMeteoTool
from beeai_framework.tools.types import ToolRunOptions
from openinference.instrumentation.beeai import BeeAIInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

 ## 1️⃣ LLM Providers: Choose Your AI Engine

BeeAI Framework supports 10+ LLM providers including Ollama, Groq, OpenAI, Watsonx.ai, and more, giving you flexibility to choose local or hosted models based on your needs. In this workshop we'll be working Ollama, so you will be running the model locally. You can find the documentation on how to connect to other providers [here](https://framework.beeai.dev/modules/backend).


### *❗* Exercise: Select your Language Model Provider

Change the `provider` and `model` variables to your desired provider and model.

If you select a provider that requires an API key URL or Project_ID, select the key icon on the left menu and set the variables to match those in the userdata.get() function.

Try several models to see how your agent performs. Note that you may need to modify the system prompt for each model, as they all have their own system prompt best practice.

In [ ]:
#Use widgets to show provider choices
providers=ipywidgets.ToggleButtons(options=['ollama','openai'])
display(providers)

In Colab, install and start Ollama for providing the Embedding Model.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

In [ ]:
provider=providers.value
from google.colab import userdata
# Ollama - No parameters required
if provider=="ollama":
    model="granite4:tiny-h"
    #model="granite3.3"
    provider_model=provider+":"+model
    !ollama pull $model
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=0))
# OpenAI - Place OpenAI API Key in Colab Secrets (key icon) as OPENAI_KEY
elif provider=="openai":
    model="gpt-5-mini"
    provider_model=provider+":"+model
    api_key = userdata.get('OPENAI_KEY')             #Set secret value using key in left menu
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=1), api_key=api_key)
else:
  print("Provider " + provider + " undefined")

# Pull Recipe and Food Stock Data into Colab

In [ ]:
if os.getcwd() == "/":
  os.chdir('/content/')
if os.path.exists("kitchen-aide"):
    !rm -rf kitchen-aide
!git clone https://github.com/KenOcheltree/kitchen-aide.git --quiet kitchen-aide
os.chdir('kitchen-aide')

## 2️⃣ Provide System Prompt

A system prompt is the foundational instruction that defines your agent's identity, role, and behavior. Think of it as the agent's "job description" and "training manual" rolled into one. Each model responds differently to the same system prompt, so experimentation is necessary.

Some key components of a strong system prompt:

- Identity: Who is the agent?
- Role: What is their function?
- Context: What environment are they operating in?
- Rules: What constraints and guidelines must they follow?
- Knowledge: What domain-specific information do they need?

### *❗* Exercise: Customize Your System Prompt
Try modifying the system prompt. Customize the "basic rules" section to add your own. Note that changes to the system prompt will affect the performance of the model. Creating a great `System Prompt` is an art, not a science.

In [ ]:
todays_date = date.today().strftime("%B %d, %Y")
instruct_prompt = f"""You help to assess ingredients needed to prepare recipes

Today's date is {todays_date}.

Tools:
- ThinkTool: Helps you plan and reason before you act. Use this tool when you need to think.
- DuckDuckGoSearchTool: Use this tool to search for recipes on the internet
- get_pantry_tool: Use this tool to gather a list of your ingredients in your pantry.
- get_fridge_tool: Use this tool to gather a list of your ingredients in your fridge.
- get_freezer_tool: Use this tool to gather a list of your ingredients in your freezer.
- my_recipe_search: your personal recipes.

Basic Rules:
- Determine the recipes that can be prepared with the ingredients that we have in our pantry, fridge or freezer
- Favor my recipes over recipes on the internet
- If we cant' prepare a recipe, make a shopping list of the items needed for a recipe
"""

## 3️⃣ Memory Systems: Maintaining Context across iterations or sessions
Why Memory Matters
Short term memory allows agents to:
- Remember previous conversation turns
- Build on past interactions
- Maintain context across multiple queries

Long Term memory allows agents to:
- Learn from user preferences
- Provided a grounded source of truth
- Pull data from non public data sources

### *❗* Exercise: Choose your memory strategy
`Memory` is an important piece of AI agents. Experiment with the different startegies by running only 1 of the cells and finishing the notebook. Optionally, once you have ran the entire notebook, come back and select a differnt memory approach and see how that affects the agent output.

In [ ]:
from beeai_framework.memory import UnconstrainedMemory

memory = UnconstrainedMemory()

## 4️⃣ Tools: Enabling LLMs to Take Action

What Are Tools?
Tools are external capabilities that extend your agent beyond just generating text. They can be API calls, code, or even calls to other AI models.

The BeeAI framework provides [built in tools](https://framework.beeai.dev/modules/tools#built-in-tools) for common tool types, but also provides the ability to create [custom tools](https://framework.beeai.dev/modules/tools#creating-custom-tools).

### Adding Framework Provided Tools
The **Think tool** encourages a Re-Act pattern where the agent reasons and plans before calling a tool.

The DuckDuckGoSearchTool is a Web Search tool that provides relevant data from the internet to the LLM

In [ ]:
think_tool = ThinkTool()
search_tool = DuckDuckGoSearchTool()

### Add Custom Tool to Read Inventory


There are 2 ways to provide custom tools to your agent. For simple tools you can use the `@tool` decorator above the function. For more complex tools, you can extend the `Tool Class` and customize things such as the run time and tool execution.  

We will create a more advanced custom tool here without the `@tool` decorator. To learn more about advanced tool customization, take a look at this section in the [documentation](https://framework.beeai.dev/modules/tools#advanced-custom-tool).

In [ ]:
class InventoryToolInput(BaseModel):
    location: str | None = Field(description="Name of the location to gather the inventory.")

class InventoryTool(Tool[InventoryToolInput, ToolRunOptions, StringToolOutput]):
    """
    Search the inventory of a kitchen location for ingredients.

    Args:
        string name of the location to gather the inventory

    Returns:
        The inventory of ingredients stored in that location
    """

    name = "Inventory"
    description = "Search the inventory of a kitchen location for ingredients."
    input_schema = InventoryToolInput

    def __init__(
            self,
        location: Optional[str] = None,
        options: dict[str, Any] | None = None
        ) -> None:
        """
        Initialize the tool.

        Args:
            location: Inventory Location
            options: Additional tool options
        """
        self.location = location
        super().__init__(options)

    def _create_emitter(self) -> Emitter:
        return Emitter.root().child(
            namespace=["tool", "example", "result"],
            creator=self,
        )

    async def _run(
        self, input: InventoryToolInput, options: ToolRunOptions | None, context: RunContext
    ) -> StringToolOutput:
        location = input.location
        print("input.location: ",input.location)
        #Add Code here to grab the inventory...
        file_path=location + "_contents.md"
        try:
          with open(file_path, 'r') as file:
            result = file.read()
        except:
             print("Error in reading the file: ", file_path, ". Please check the file name.")
        print(location, " inventory:", result)
        return StringToolOutput(result=result)


## 5️⃣ Creating a RAG (Retrieval Augmented Generation) Tool to Search My Recipes

`RAG` (Retrieval-Augmented Generation) is “search + write”: you ask a question, the system retrieves the most relevant snippets from an indexed knowledge base (via embeddings) and the model composes an answer grounded in those snippets.

The BeeAI Framework has built in abstractions to make RAG simple to implement. Read more about it [here](https://framework.beeai.dev/modules/rag).

First, we must pull an embedding model which converts text into numerical vectors so we can compare meanings and retrieve the most relevant snippets. The original document is:
1. preprocessed (cleaned + broken into chunks)
2. ran through the embedding algorithm
3. stored in the vector database


In [ ]:
!ollama pull nomic-embed-text:latest

In [ ]:
embedding_model = EmbeddingModel.from_name("ollama:nomic-embed-text")

### *❗* Exercise: Internal documents
Take a look at the internal documents so you know what type of questions to ask your agent


In [ ]:
file_path="my_recipes.md"
with open(file_path, 'r') as file:
    content = file.read()

#Print first 50 lines
lines = content.splitlines()
for i in range(min(50, len(lines))):
    print(lines[i])

Load the document using the `DocumentLoader` and split the document into chunks using the `text_splitter`.

In [ ]:
loader = DocumentLoader.from_name(
    name="langchain:UnstructuredMarkdownLoader", file_path=file_path
)
try:
    documents = await loader.load()
except Exception as e:
    print(f"Failed to load documents: {e}")

# Split documents into chunks
text_splitter = TextSplitter.from_name(
    name="langchain:RecursiveCharacterTextSplitter", chunk_size=1000, chunk_overlap=200
)
try:
    documents = await text_splitter.split_documents(documents)
except Exception as e:
    print(f"Failed to split documents: {e}")
print(f"Loaded {len(documents)} document chunks")

Create the `TemporalVectorStore`, which means this vector store also tracks time.

In [ ]:
# Create vector store and add documents
vector_store = VectorStore.from_name(name="beeai:TemporalVectorStore", embedding_model=embedding_model)
await vector_store.add_documents(documents=documents)
print("Vector store populated with documents")

Create the `internal_document_search` tool! Because the `VectorStoreSearchTool` is a built in tool wrapper, we don't need to use the `@tool` decorator or extend the custom `Tool class`.

In [ ]:
# Create the vector store search tool
my_recipe_search = VectorStoreSearchTool(vector_store=vector_store)

## Explore Observability: See what is happening under the hood

Create the function that sets up observability using `OpenTelemetry` and [Arize's Phoenix Platform](https://arize.com/docs/phoenix/inferences/how-to-inferences/manage-the-app). There a several ways to view what is happening under the hood of your agent. View the observability documentation [here](https://framework.beeai.dev/modules/observability).

In [ ]:
def setup_observability(endpoint: str = "http://localhost:6006/v1/traces") -> None:
    """
    Sets up OpenTelemetry with OTLP HTTP exporter and instruments the beeai framework.
    """
    resource = Resource(attributes={})
    tracer_provider = trace_sdk.TracerProvider(resource=resource)
    tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
    trace_api.set_tracer_provider(tracer_provider)

    BeeAIInstrumentor().instrument()

In [ ]:
load_dotenv()
# Enable OpenTelemetry integration
setup_observability("http://localhost:6006/v1/traces")
px_session = px.launch_app()

## 6️⃣ Conditional Requirements: Guiding Agent Behavior


What Are Conditional Requirements?
[Conditional requirements](https://framework.beeai.dev/experimental/requirement-agent#conditional-requirement) ensure your agents are reliable by controlling when and how tools are used. They're like business rules for agent behavior. You can make them as strict (esentially writing a static workflow) or flexible (no rules! LLM decides) as you'd like.

The rules that you enforce may seem simple in the BeeAI framework, but in other frameworks they require ~5X the amount of code. Check out this [blog](https://beeai.dev/blog/reliable-ai-agents) where we built the same agent in BeeAI and other agent framework LangGraph.

These conditional requirements enforce the following in only 3 lines of code:
1. The agent must call the think tool as the first tool call. It is not allowed to call it consecutive times in a row.
2. The wikipedia_tool can only be called after the think tool, but not consecutively. It has a relative priority of 10.
3. The DuckDuckGo Internet search tool can also only be called after the Think tool, it is allowed to be called up to 3 times, it must be invoked at least once, and it has a relative priority of 15.
4. The internal_document_search tool can only be called after the think tool, it is allowed to be called multiple times in a row, it must be called at least once, and it has a relative priority of 20.



##  7️⃣ Assemble Your Reliable BeeAI Agent

This is the part we've been working towards! Let's assemble the agent with all the parts we created.

In [ ]:
get_pantry_tool=InventoryTool(location="pantry")
get_freezer_tool=InventoryTool(location="freezer")
get_fridge_tool=InventoryTool(location="fridge")

agent = RequirementAgent(
    llm=llm,
    instructions= instruct_prompt,
    memory = memory,
    tools=[ThinkTool(), DuckDuckGoSearchTool(), get_pantry_tool, get_freezer_tool, get_fridge_tool, my_recipe_search],
    requirements=[
        ConditionalRequirement(ThinkTool, consecutive_allowed=False, force_at_step=1 ),
        ConditionalRequirement(get_pantry_tool, force_at_step=2 ),
        ConditionalRequirement(get_freezer_tool, force_at_step=3 ),
        ConditionalRequirement(get_fridge_tool, force_at_step=4 ),
        ConditionalRequirement(my_recipe_search, force_at_step=5 ),
        ConditionalRequirement(DuckDuckGoSearchTool, only_after=my_recipe_search),
    ],
    # Log intermediate steps to the console
    middlewares=[GlobalTrajectoryMiddleware(included=[Tool])],
)

In [ ]:
#px_session.view()

### *❗* Exercise: Test Your Agent
Change the execution settings and see what happens. Does your agent run out of iterations? Every task is different and its important to balance flexibility with control.

Example Questions:
- Which of my recipes can I prepare with the items I have on hand?
- What do I need to buy to make blueberry scones?
- What breakfast recipes can I make with the items I have on hand?
- What internet recipe can I prepare with the items I have on hand?
- What items do I need to buy to prepare my pizza recipe?

In [ ]:
while (question := input("Enter what you would like to do (Enter exit to end): ")) !="exit":
    response = await agent.run(question, max_retries_per_step=3, total_max_retries=25)
    print(response.last_message.text)